In [ ]:

def joa(symbol):
    
    import math
    import pandas_datareader as web
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import MinMaxScaler
    from keras.models import Sequential
    from keras.layers import Dense, LSTM
    import matplotlib.pyplot as plt
    plt.style.use('fivethirtyeight')
    import plotly.express as px

    #Get the stock quote 
    df = web.DataReader(symbol, data_source='yahoo', start='2015-07-30', end='2021-07-30') 

    #show data
    #df.tail(10)

    # get the number of rows and colums inthe data set
    #df.shape



    import plotly.express as px

  

    #Create a new dataframe with only the 'Close' column
    data = df.filter(['Close'])
    #Converting the dataframe to a numpy array
    dataset = data.values
    #Get /Compute the number of rows to train the model on
    training_data_len = math.ceil( len(dataset) *.8) # 80% training date. Math.ceil is to round up the numbers

    # Using MinMaxScaler from sklearn to scale the data betweek 0 and 1, preprocess date to fit neural network
    scaler = MinMaxScaler(feature_range = (0,1))
    scaled_data = scaler.fit_transform(dataset)

    # create the training datae set
    #create the scaled training date set
    train_data = scaled_data[0:training_data_len, :]

    #Split the data into x_train and y_train data sets
    x_train=[]
    y_train = []
    for i in range(60,len(train_data)):
        x_train.append(train_data[i-60:i,0])
        y_train.append(train_data[i,0])
        if i<= 61:
            print(x_train)
            print(y_train)
            print()


    #Convert x_train and y_train to numpy arrays
    x_train, y_train = np.array(x_train), np.array(y_train)

    #Reshape the data into the shape accepted by the LSTM
    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
    #x_train.shape

    #Build the LSTM network model
    model = Sequential()

    # Layer 1 
    model.add(LSTM(units=50, return_sequences=True,input_shape=(x_train.shape[1],1)))

    # Layer 2
    model.add(LSTM(units=50, return_sequences=False))

    #Output layers
    model.add(Dense(units=25))
    model.add(Dense(units=1))

    #Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Summarize the model
    #model.summary()

    # Training the model
    model.fit(x_train, y_train, batch_size = 1, epochs = 1)

    # Create the testing data set
    # Crete a new array containing scaled valueds 20% of the dataset
    test_data = scaled_data[training_data_len -60:,:]

    x_test = []
    y_test =  dataset[training_data_len : , : ] 
    
    for i in range(60,len(test_data)):
        x_test.append(test_data[i-60:i,0])

    # convert the data to a numpy array
    x_test = np.array(x_test)

    #Reshape the data into the shape accepted by the LSTM
    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

    #Getting the models predicted price values
    predictions = model.predict(x_test) 
    predictions = scaler.inverse_transform(predictions)#Undo scaling

    # Get the root mean squrared erro(RMSE)--how accurate the model to predit the response, the standard deviatation of the residules. 
    rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
    rmse

    #Plot/Create the data for the graph
    train = data[:training_data_len].copy()
    valid = data[training_data_len:].copy()
    valid['Predictions'] = predictions

    import plotly.graph_objects as go

    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Line(x=train.index, y=train['Close'],
                        mode='lines',
                        name='Train'))
    fig.add_trace(go.Line(x=valid.index, y=valid['Close'],
                        mode='lines',
                        name='Val'))
    fig.add_trace(go.Line(x=valid.index, y=valid['Predictions'],
                        mode='lines',
                        name='Predictions'))
    fig.update_layout(title='Model',
                       xaxis_title='Date',
                       yaxis_title='Close Price USD ($)')
    #fig.show()
    #valid.plot()

    #fig = px.line(valid[['Close', 'Predictions']])
    #fig.show()
    return valid
    


In [ ]:
import joaa
df = joaa.joa('aapl')
df

In [ ]:
# Dash Framework - stock price visualization demo
import pandas_datareader.data as web
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import datetime

# First the layout
app = dash.Dash()
app.title = "Stock Visualization"
app.layout = html.Div(children=[
    html.H1('Stock Visualization Dashboard'),
    dcc.Input(id="input", value='', type='text'),
    html.Div(id="output-graph")
])

# Second the interaction with the user


@app.callback(
    Output(component_id="output-graph", component_property='children'),
    [Input(component_id="input", component_property="value")]
)
def update_value(input_data):
    import plotly.express as px
    import joaa
    df = joaa.joa(input_data).reset_index()
    #df = px.data.stocks()
    fig = px.line(df, x='Date', y=['Close', 'Predictions'])
    #fig.show()
    return dcc.Graph(id="demo", figure= fig)


if __name__ == "__main__":
    app.run_server(debug=False, port=8058)

Dash is running on http://127.0.0.1:8058/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8058/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Aug/2021 20:51:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2021 20:51:15] "GET /_dash-component-suites/dash/deps/polyfill@7.v1_21_0m1627149480.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2021 20:51:15] "GET /_dash-component-suites/dash/deps/react@16.v1_21_0m1627149480.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2021 20:51:15] "GET /_dash-component-suites/dash/deps/prop-types@15.v1_21_0m1627149480.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2021 20:51:15] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_17_1m1627149480.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2021 20:51:15] "GET /_dash-component-suites/dash/deps/react-dom@16.v1_21_0m1627149480.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2021 20:51:15] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_17_1m1627149480.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2021 20:51:16] "GET /

[array([0.18240346, 0.20100143, 0.1986171 , 0.19933238, 0.20577016,
       0.20410112, 0.22484501, 0.24177393, 0.23962801, 0.23795896,
       0.24654267, 0.26180255, 0.26394848, 0.25584165, 0.24320458,
       0.22007628, 0.19980927, 0.17501192, 0.15665234, 0.16905102,
       0.1840725 , 0.19551741, 0.19384837, 0.15665234, 0.1549833 ,
       0.15689083, 0.14830713, 0.15689083, 0.15951355, 0.14997617,
       0.14377679, 0.14997617, 0.17334287, 0.17930377, 0.16833574,
       0.15283737, 0.14997617, 0.14234623, 0.13710061, 0.13566996,
       0.14067719, 0.12064857, 0.12112545, 0.13233188, 0.12660947,
       0.13948493, 0.16642821, 0.16738198, 0.17501192, 0.20195519,
       0.20410112, 0.20577016, 0.18860275, 0.1900334 , 0.18597994,
       0.18216497, 0.18121121, 0.17787312, 0.16595133, 0.1871721 ])]
[0.11325702726498776]

[array([0.18240346, 0.20100143, 0.1986171 , 0.19933238, 0.20577016,
       0.20410112, 0.22484501, 0.24177393, 0.23962801, 0.23795896,
       0.24654267, 0.26180255, 0.26

127.0.0.1 - - [01/Aug/2021 20:51:25] "POST /_dash-update-component HTTP/1.1" 500 -


1150/1150 [==============================] - 43s 19ms/step - loss: 0.0067


...

C:\Users\Joanne\anaconda3\envs\hvplot_dev\lib\site-packages\plotly\graph_objs\_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


127.0.0.1 - - [01/Aug/2021 20:54:06] "POST /_dash-update-component HTTP/1.1" 200 -


1093/1150 [===========================>..] - ETA: 1s - loss: 0.0092

C:\Users\Joanne\anaconda3\envs\hvplot_dev\lib\site-packages\plotly\graph_objs\_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


127.0.0.1 - - [01/Aug/2021 20:54:07] "POST /_dash-update-component HTTP/1.1" 200 -


1150/1150 [==============================] - 37s 23ms/step - loss: 0.0090 0s - loss: 0.0


C:\Users\Joanne\anaconda3\envs\hvplot_dev\lib\site-packages\plotly\graph_objs\_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


127.0.0.1 - - [01/Aug/2021 20:54:09] "POST /_dash-update-component HTTP/1.1" 200 -
